# CH01-01: Poetry 安裝與配置

**課程目標:**
- 理解 Poetry 的核心功能與優勢
- 掌握 Poetry 的安裝與基本配置
- 學會使用 Poetry 管理 NLP 專案依賴
- 了解 pyproject.toml 的結構與配置選項

**學習時間:** 約 60 分鐘

**前置知識:**
- Python 基礎語法
- 命令列基本操作
- pip 套件管理基礎

---

## 📚 目錄

1. [為什麼需要 Poetry?](#1)
2. [Poetry 安裝 (跨平台)](#2)
3. [初始化 NLP 專案](#3)
4. [依賴管理實戰](#4)
5. [pyproject.toml 深度解析](#5)
6. [poetry.lock 鎖定機制](#6)
7. [虛擬環境管理](#7)
8. [實戰練習](#8)

---

In [ ]:
# 環境設定與套件導入
import subprocess
import sys
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定顯示風格
sns.set_style('whitegrid')

print("✅ 環境設定完成")
print(f"Python 版本: {sys.version.split()[0]}")
print(f"工作目錄: {os.getcwd()}")

<a id="1"></a>
## 1. 為什麼需要 Poetry?

### 1.1 傳統依賴管理的痛點

在 Poetry 出現之前,Python 開發者面臨**三大挑戰**:

1. **依賴地獄 (Dependency Hell)**
   - 手動管理 `requirements.txt`,容易版本衝突
   - pip 無法自動解決依賴衝突

2. **環境不一致**
   - 開發環境與生產環境套件版本不同
   - 經典問題:「在我機器上可以跑」

3. **配置檔案混亂**
   - `setup.py`, `setup.cfg`, `MANIFEST.in`, `requirements.txt` 等多個檔案
   - 維護成本高,容易出錯

---

### 1.2 Poetry 的革命性突破

**Poetry = 依賴管理 + 虛擬環境 + 打包工具**

#### Poetry 的三大核心功能:

1. ✅ **自動依賴解析** - 使用 SAT Solver 算法自動解決版本衝突
2. ✅ **確定性鎖定** - `poetry.lock` 確保環境 100% 可重現
3. ✅ **單一配置檔案** - `pyproject.toml` 統一管理所有設定

---

In [ ]:
# 視覺化: pip vs Poetry 對比
comparison_data = {
    '特性': ['配置檔案', '依賴鎖定', '依賴解析', '虛擬環境', '打包發布', '開發依賴分離'],
    'pip + virtualenv': ['多個檔案', '手動 freeze', '不解決衝突', '手動創建', '需 setup.py', 'requirements-dev.txt'],
    'Poetry': ['pyproject.toml', 'poetry.lock', '自動解析', '自動創建', 'poetry build', '[tool.poetry.dev-dependencies]']
}

df = pd.DataFrame(comparison_data)

# 創建對比表格視覺化
fig, ax = plt.subplots(figsize=(14, 6))
ax.axis('tight')
ax.axis('off')

# 創建表格
table = ax.table(cellText=df.values, colLabels=df.columns,
                cellLoc='left', loc='center',
                colWidths=[0.25, 0.35, 0.35])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# 設定表頭樣式
for i in range(len(df.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 設定交替行顏色
for i in range(1, len(df) + 1):
    for j in range(len(df.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#F7F7F7')
        else:
            table[(i, j)].set_facecolor('white')

plt.title('pip vs Poetry 功能對比', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 核心差異:")
print("  - pip: 簡單但需手動管理,容易出錯")
print("  - Poetry: 自動化、智能化,適合生產環境")

### 1.3 Poetry vs pip 實際案例

**情境:** 專案需要 Package A (依賴 C >= 1.0, < 2.0) 和 Package B (依賴 C >= 1.5, < 3.0)

#### pip 的行為 (不確定性):
```bash
# 安裝順序 1
pip install package-a  # 安裝 C==1.9
pip install package-b  # 檢查 C==1.9 符合 >= 1.5,不動作 ✅

# 安裝順序 2
pip install package-b  # 安裝 C==2.5
pip install package-a  # 檢查 C==2.5 不符合 < 2.0,衝突! ❌
```

#### Poetry 的行為 (確定性):
```bash
poetry add package-a package-b
# 自動計算交集: C >= 1.5, < 2.0
# 選擇最新版本: C==1.9
# 寫入 poetry.lock,鎖定版本
# 無論安裝順序,結果永遠相同! ✅
```

---

<a id="2"></a>
## 2. Poetry 安裝 (跨平台)

### 2.1 Windows 安裝

#### 方法 1: 官方安裝腳本 (推薦)

```powershell
# PowerShell (以系統管理員身分執行)
(Invoke-WebRequest -Uri https://install.python-poetry.org -UseBasicParsing).Content | python -

# 設定環境變數
$Env:Path += ";$Env:APPDATA\Python\Scripts"

# 驗證安裝
poetry --version
```

#### 方法 2: pipx 安裝 (隔離環境)

```powershell
# 安裝 pipx
python -m pip install --user pipx
python -m pipx ensurepath

# 用 pipx 安裝 Poetry
pipx install poetry
```

---

### 2.2 macOS/Linux 安裝

```bash
# 官方安裝腳本
curl -sSL https://install.python-poetry.org | python3 -

# 將 Poetry 加入 PATH
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc
source ~/.bashrc

# macOS 使用 Homebrew
brew install poetry

# 驗證安裝
poetry --version
```

---

In [ ]:
# 檢查 Poetry 是否已安裝
def check_poetry_installation():
    """檢查 Poetry 安裝狀態"""
    try:
        result = subprocess.run(['poetry', '--version'], 
                              capture_output=True, text=True, check=True)
        version = result.stdout.strip()
        print(f"✅ Poetry 已安裝: {version}")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("❌ Poetry 未安裝")
        print("\n請依照上述方法安裝 Poetry:")
        print("  Windows: (Invoke-WebRequest -Uri https://install.python-poetry.org -UseBasicParsing).Content | python -")
        print("  macOS/Linux: curl -sSL https://install.python-poetry.org | python3 -")
        return False

# 執行檢查
check_poetry_installation()

# 顯示 Poetry 配置
try:
    result = subprocess.run(['poetry', 'config', '--list'], 
                          capture_output=True, text=True, check=True)
    print("\n📋 當前 Poetry 配置:")
    for line in result.stdout.strip().split('\n')[:5]:  # 只顯示前 5 項
        print(f"  {line}")
except:
    print("\n⚠️ 無法讀取 Poetry 配置")

### 2.3 Poetry 基礎配置

安裝完成後,建議進行以下配置優化:

In [ ]:
# Poetry 基礎配置 (在終端執行以下指令)
config_commands = [
    "poetry config virtualenvs.in-project true",  # 將 .venv 建立在專案目錄內
    "poetry config installer.max-workers 10",     # 平行下載加速
]

print("🔧 建議的 Poetry 配置:")
print("\n請在終端執行以下指令:\n")
for i, cmd in enumerate(config_commands, 1):
    print(f"{i}. {cmd}")

print("\n配置說明:")
print("  - virtualenvs.in-project: 方便 IDE 辨識虛擬環境")
print("  - installer.max-workers: 提升套件下載速度")

# 視覺化配置項目
config_data = {
    '配置項目': ['virtualenvs.in-project', 'installer.max-workers', 'virtualenvs.create'],
    '建議值': ['true', '10', 'true'],
    '說明': ['虛擬環境建在專案內', '平行下載數', '自動創建虛擬環境']
}

df_config = pd.DataFrame(config_data)

fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_config.values, colLabels=df_config.columns,
                cellLoc='left', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# 設定表頭樣式
for i in range(len(df_config.columns)):
    table[(0, i)].set_facecolor('#FF6B6B')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.title('Poetry 核心配置項目', fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

<a id="3"></a>
## 3. 初始化 NLP 專案

### 3.1 創建新專案

#### 方法 1: 使用 `poetry new` (推薦給新專案)

```bash
poetry new nlp-project

# 專案結構:
nlp-project/
├── pyproject.toml         # 專案配置檔案
├── README.md              # 專案說明
├── nlp_project/           # 原始碼目錄
│   └── __init__.py
└── tests/                 # 測試目錄
    └── __init__.py
```

#### 方法 2: 使用 `poetry init` (現有專案)

```bash
cd my-existing-project
poetry init  # 互動式初始化
```

---

In [ ]:
# 模擬專案結構視覺化
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# 專案根目錄
root_rect = mpatches.FancyBboxPatch((1, 7), 8, 1.5, 
                                     boxstyle="round,pad=0.1", 
                                     linewidth=2, edgecolor='#4ECDC4', 
                                     facecolor='#E8F8F5')
ax.add_patch(root_rect)
ax.text(5, 7.75, '📁 nlp-project/', ha='center', fontsize=14, fontweight='bold')

# 檔案與資料夾
items = [
    (2, 5.5, '📄 pyproject.toml', '#FFE66D'),
    (2, 4.5, '📄 README.md', '#FFE66D'),
    (2, 3.5, '📄 poetry.lock', '#FFE66D'),
    (6, 5.5, '📁 nlp_project/', '#E3F2FD'),
    (6, 4.5, '📁 tests/', '#E3F2FD'),
    (6, 3.5, '📁 .venv/', '#F3E5F5'),
]

for x, y, text, color in items:
    rect = mpatches.FancyBboxPatch((x-0.9, y-0.3), 3.5, 0.6,
                                    boxstyle="round,pad=0.05",
                                    linewidth=1, edgecolor='gray',
                                    facecolor=color)
    ax.add_patch(rect)
    ax.text(x+0.85, y, text, ha='center', va='center', fontsize=11)

# 說明文字
ax.text(1, 2, '核心配置檔案', fontsize=10, color='#666')
ax.text(1, 1.5, '原始碼與測試', fontsize=10, color='#666')
ax.text(1, 1, '虛擬環境 (自動創建)', fontsize=10, color='#666')

# 箭頭指示
ax.annotate('', xy=(2, 5.5), xytext=(1.5, 2),
            arrowprops=dict(arrowstyle='->', lw=1, color='#999'))
ax.annotate('', xy=(6, 5), xytext=(1.5, 1.5),
            arrowprops=dict(arrowstyle='->', lw=1, color='#999'))
ax.annotate('', xy=(6, 3.5), xytext=(1.5, 1),
            arrowprops=dict(arrowstyle='->', lw=1, color='#999'))

plt.title('Poetry 專案標準結構', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📁 專案結構說明:")
print("  pyproject.toml - Poetry 核心配置檔案 (依賴、腳本、打包設定)")
print("  poetry.lock - 依賴版本鎖定檔案 (確保環境一致性)")
print("  nlp_project/ - 原始碼目錄 (Python 套件)")
print("  tests/ - 測試程式碼目錄")
print("  .venv/ - 虛擬環境 (Poetry 自動創建)")

<a id="4"></a>
## 4. 依賴管理實戰

### 4.1 安裝 NLP 核心套件

#### 基本安裝指令:

```bash
# 進入專案目錄
cd nlp-project

# 啟動虛擬環境
poetry shell

# 安裝中文 NLP 工具
poetry add jieba

# 安裝英文 NLP 工具
poetry add nltk spacy

# 安裝深度學習 NLP
poetry add transformers datasets tokenizers

# 安裝深度學習框架 (二選一)
poetry add torch torchvision torchaudio  # PyTorch
# 或
poetry add tensorflow  # TensorFlow
```

---

### 4.2 開發依賴管理

```bash
# 安裝開發依賴 (不會打包到生產環境)
poetry add --group dev pytest pytest-cov
poetry add --group dev black flake8 mypy
poetry add --group dev jupyter ipykernel
```

---

In [ ]:
# Poetry 常用指令速查表
commands_data = {
    '指令': [
        'poetry add <package>',
        'poetry add --group dev <package>',
        'poetry install',
        'poetry update',
        'poetry remove <package>',
        'poetry show',
        'poetry show --tree',
        'poetry run python script.py',
        'poetry shell',
        'poetry lock'
    ],
    '功能說明': [
        '安裝套件並加入依賴',
        '安裝開發依賴',
        '根據 lock 檔安裝所有依賴',
        '更新所有套件到最新版本',
        '移除套件',
        '顯示已安裝套件',
        '顯示依賴樹狀結構',
        '在虛擬環境中執行腳本',
        '啟動虛擬環境 Shell',
        '更新 lock 檔 (不安裝)'
    ],
    '使用頻率': ['⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐', 
              '⭐⭐⭐', '⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐']
}

df_commands = pd.DataFrame(commands_data)

fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_commands.values, colLabels=df_commands.columns,
                cellLoc='left', loc='center',
                colWidths=[0.35, 0.45, 0.2])

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2.2)

# 設定表頭樣式
for i in range(len(df_commands.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 設定交替行顏色
for i in range(1, len(df_commands) + 1):
    for j in range(len(df_commands.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#F0F0F0')

plt.title('Poetry 常用指令速查表', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 使用技巧:")
print("  1. 優先使用 'poetry add' 而非 'pip install' - 自動更新 pyproject.toml")
print("  2. 定期執行 'poetry update' - 保持套件最新且相容")
print("  3. 使用 'poetry show --tree' - 檢查依賴關係,發現衝突")
print("  4. 提交 poetry.lock 到版控 - 確保團隊環境一致")

<a id="5"></a>
## 5. pyproject.toml 深度解析

### 5.1 pyproject.toml 結構

`pyproject.toml` 是 Poetry 的核心配置檔案,採用 TOML 格式 (Tom's Obvious, Minimal Language)。

#### 完整 NLP 專案範例:

In [ ]:
# 完整的 pyproject.toml 範例
pyproject_example = """
[tool.poetry]
name = "ispan-nlp-project"
version = "1.0.0"
description = "iSpan NLP 課程專案"
authors = ["iSpan Team <team@ispan.com>"]
license = "MIT"
readme = "README.md"

[tool.poetry.dependencies]
python = "^3.10"

# 中文 NLP 工具
jieba = "^0.42.1"

# 英文 NLP 工具
nltk = "^3.8.1"
spacy = "^3.7.2"

# 深度學習 NLP
transformers = "^4.35.0"
datasets = "^2.14.0"
tokenizers = "^0.15.0"

# 深度學習框架 (條件依賴)
torch = {version = "^2.1.0", optional = true}
tensorflow = {version = "^2.14.0", optional = true}

# 工具套件
numpy = "^1.24.0"
pandas = "^2.1.0"
scikit-learn = "^1.3.0"

[tool.poetry.extras]
pytorch = ["torch", "torchvision"]
tensorflow = ["tensorflow"]

[tool.poetry.group.dev.dependencies]
pytest = "^7.4.3"
black = "^23.11.0"
jupyter = "^1.0.0"

[tool.poetry.scripts]
nlp-train = "nlp_project.train:main"

[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"
"""

print("📄 完整的 pyproject.toml 範例:")
print("="*60)
print(pyproject_example)
print("="*60)

print("\n🔍 關鍵區塊說明:")
print("\n[tool.poetry] - 專案基本資訊")
print("  - name: 專案名稱 (發布到 PyPI 時使用)")
print("  - version: 版本號 (遵循語義化版本)")
print("  - description: 專案描述")
print("  - authors: 作者資訊")

print("\n[tool.poetry.dependencies] - 生產依賴")
print("  - python = \"^3.10\": Python 版本約束")
print("  - ^0.42.1: >= 0.42.1, < 1.0.0 (允許 minor 更新)")
print("  - optional = true: 條件依賴 (需透過 extras 安裝)")

print("\n[tool.poetry.extras] - 可選功能組")
print("  - 安裝方式: poetry install --extras pytorch")

print("\n[tool.poetry.group.dev.dependencies] - 開發依賴")
print("  - 不會打包到生產環境")
print("  - 包含測試、格式化、Notebook 等工具")

print("\n[tool.poetry.scripts] - 命令列工具")
print("  - 定義可執行腳本")
print("  - 安裝後可直接執行: nlp-train")

### 5.2 版本約束語法

Poetry 使用語義化版本 (Semantic Versioning) 的約束語法:

In [ ]:
# 版本約束語法說明
version_data = {
    '約束語法': ['^1.2.3', '~1.2.3', '>=1.2.3', '==1.2.3', '1.2.*', '>1.2,<2.0'],
    '等同於': ['>=1.2.3, <2.0.0', '>=1.2.3, <1.3.0', '>=1.2.3', '==1.2.3', 
              '>=1.2.0, <1.3.0', '>1.2.0, <2.0.0'],
    '說明': ['允許 minor/patch 更新', '只允許 patch 更新', '大於等於', 
            '精確版本', '通配符', '複合條件'],
    '推薦度': ['⭐⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐', '⭐', '⭐⭐', '⭐⭐⭐']
}

df_version = pd.DataFrame(version_data)

fig, ax = plt.subplots(figsize=(14, 5))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_version.values, colLabels=df_version.columns,
                cellLoc='left', loc='center',
                colWidths=[0.2, 0.3, 0.35, 0.15])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭樣式
for i in range(len(df_version.columns)):
    table[(0, i)].set_facecolor('#FF6B6B')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 高亮推薦項
table[(1, 0)].set_facecolor('#FFEB3B')  # ^ 語法

plt.title('Poetry 版本約束語法', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 最佳實踐:")
print("  - 優先使用 '^' (caret) 語法 - 平衡穩定性與更新")
print("  - 避免使用 '==' - 過於嚴格,難以解決依賴衝突")
print("  - 生產環境依賴 poetry.lock - 版本已完全鎖定")

print("\n🔢 語義化版本格式: MAJOR.MINOR.PATCH")
print("  - MAJOR: 重大改版,不向後相容")
print("  - MINOR: 新增功能,向後相容")
print("  - PATCH: Bug 修復,向後相容")

<a id="6"></a>
## 6. poetry.lock 鎖定機制

### 6.1 為什麼需要 poetry.lock?

`poetry.lock` 是依賴版本的**快照檔案**,確保所有環境使用完全相同的套件版本。

#### 鎖定機制的核心價值:

1. **可重現性 (Reproducibility)**
   - 開發環境 = 測試環境 = 生產環境
   - 消除「在我機器上可以跑」問題

2. **完整性驗證 (Integrity Check)**
   - 每個套件包含 SHA-256 雜湊值
   - 防止套件被篡改或替換

3. **依賴解析快取 (Resolution Cache)**
   - 不需每次重新計算依賴
   - 大幅提升安裝速度

---

In [ ]:
# poetry.lock 檔案結構示例
lock_example = """
[[package]]
name = "jieba"
version = "0.42.1"
description = "Chinese text segmentation"
category = "main"
optional = false
python-versions = "*"
files = [
    {file = "jieba-0.42.1-py2.py3-none-any.whl", hash = "sha256:xxx"},
    {file = "jieba-0.42.1.tar.gz", hash = "sha256:yyy"},
]

[metadata]
lock-version = "2.0"
python-versions = "^3.10"
content-hash = "abc123..."
"""

print("📄 poetry.lock 檔案結構示例:")
print("="*60)
print(lock_example)
print("="*60)

print("\n🔍 關鍵欄位說明:")
print("  name & version: 精確的套件名稱與版本")
print("  files[].hash: SHA-256 雜湊值 (防篡改)")
print("  content-hash: 整個依賴樹的雜湊值 (快速比對)")

# 視覺化鎖定流程
fig, ax = plt.subplots(figsize=(14, 6))
ax.set_xlim(0, 10)
ax.set_ylim(0, 8)
ax.axis('off')

# 流程步驟
steps = [
    (1, 6, 'pyproject.toml\n版本約束', '#FFE66D'),
    (5, 6, 'SAT Solver\n依賴解析', '#4ECDC4'),
    (1, 3, 'poetry.lock\n精確版本', '#FF6B6B'),
    (5, 3, '安裝套件\n驗證雜湊', '#95E1D3'),
    (8, 4.5, '環境一致\n✅', '#F38181')
]

for x, y, text, color in steps:
    rect = mpatches.FancyBboxPatch((x-0.8, y-0.5), 2, 1,
                                    boxstyle="round,pad=0.1",
                                    linewidth=2, edgecolor='gray',
                                    facecolor=color)
    ax.add_patch(rect)
    ax.text(x+0.2, y, text, ha='center', va='center', fontsize=11, fontweight='bold')

# 箭頭
ax.annotate('', xy=(4, 6), xytext=(2.2, 6),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))
ax.annotate('', xy=(5, 5.5), xytext=(5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))
ax.annotate('', xy=(4, 3), xytext=(2.2, 3),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))
ax.annotate('', xy=(6.8, 3.5), xytext=(6, 3.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

plt.title('Poetry 依賴鎖定流程', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n⚠️ 重要提醒:")
print("  1. 務必將 poetry.lock 提交到版本控制 (Git)")
print("  2. 團隊成員使用 'poetry install' (而非 add) 確保版本一致")
print("  3. 遇到 lock 衝突,執行 'poetry lock --no-update' 重新鎖定")

### 6.2 poetry.lock 工作流程

#### 開發者 A (首次創建):
```bash
poetry add jieba  # 安裝套件
# → 更新 pyproject.toml
# → 解析依賴,生成 poetry.lock
git add pyproject.toml poetry.lock
git commit -m "feat: add jieba dependency"
git push
```

#### 開發者 B (同步環境):
```bash
git pull
poetry install  # 根據 lock 檔安裝
# → 完全相同的套件版本
# → 驗證雜湊值,確保完整性
```

---

<a id="7"></a>
## 7. 虛擬環境管理

### 7.1 Poetry 虛擬環境機制

Poetry 自動創建並管理虛擬環境,無需手動操作 `virtualenv` 或 `venv`。

#### 虛擬環境配置:

In [ ]:
# 虛擬環境管理指令
venv_commands = {
    '指令': [
        'poetry env use python3.10',
        'poetry env info',
        'poetry env list',
        'poetry env remove python3.10',
        'poetry shell',
        'poetry run python',
        'exit'
    ],
    '功能': [
        '指定 Python 版本創建環境',
        '顯示當前環境資訊',
        '列出所有虛擬環境',
        '刪除指定虛擬環境',
        '啟動虛擬環境 Shell',
        '在虛擬環境執行指令',
        '退出虛擬環境'
    ]
}

df_venv = pd.DataFrame(venv_commands)

fig, ax = plt.subplots(figsize=(12, 6))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_venv.values, colLabels=df_venv.columns,
                cellLoc='left', loc='center',
                colWidths=[0.5, 0.5])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭
for i in range(len(df_venv.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.title('Poetry 虛擬環境管理指令', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔧 虛擬環境最佳實踐:")
print("  1. 設定 virtualenvs.in-project = true")
print("     → .venv 建立在專案目錄,方便 IDE 辨識")
print("  2. 使用 'poetry shell' 進入環境")
print("     → 自動啟動,無需記憶路徑")
print("  3. 或使用 'poetry run' 執行單一指令")
print("     → 無需進入 shell,適合 CI/CD")

In [ ]:
# 檢查當前虛擬環境狀態
def check_virtual_env():
    """檢查虛擬環境狀態"""
    print("🔍 當前 Python 環境資訊:\n")
    print(f"Python 執行檔路徑: {sys.executable}")
    print(f"Python 版本: {sys.version.split()[0]}")
    
    # 檢查是否在虛擬環境中
    in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
    
    if in_venv:
        print(f"\n✅ 正在虛擬環境中")
        print(f"環境路徑: {sys.prefix}")
    else:
        print(f"\n⚠️ 未在虛擬環境中 (使用系統 Python)")
        print("建議執行: poetry shell")
    
    # 列出已安裝套件數量
    try:
        import pkg_resources
        installed_packages = list(pkg_resources.working_set)
        print(f"\n已安裝套件數量: {len(installed_packages)}")
    except:
        pass

check_virtual_env()

### 7.2 虛擬環境位置設定

Poetry 提供兩種虛擬環境存放策略:

#### 策略 1: 集中管理 (預設)
```bash
# Linux/macOS: ~/.cache/pypoetry/virtualenvs/
# Windows: %LOCALAPPDATA%\pypoetry\Cache\virtualenvs
```

#### 策略 2: 專案目錄內 (推薦)
```bash
poetry config virtualenvs.in-project true
# → 在專案根目錄創建 .venv/
```

**推薦使用策略 2 的理由:**
- IDE (VS Code, PyCharm) 自動辨識
- 便於 Docker 映像構建
- 刪除專案時一併清除環境

---

<a id="8"></a>
## 8. 實戰練習

### 練習 1: 創建完整 NLP 專案

根據以下步驟,創建一個完整的 NLP 專案:

In [ ]:
# 練習 1: 專案創建步驟
exercise1_steps = """
步驟 1: 創建新專案
  poetry new ispan-nlp-demo
  cd ispan-nlp-demo

步驟 2: 配置 Poetry
  poetry config virtualenvs.in-project true

步驟 3: 安裝中文 NLP 工具
  poetry add jieba

步驟 4: 安裝英文 NLP 工具
  poetry add nltk spacy

步驟 5: 安裝深度學習套件
  poetry add transformers torch

步驟 6: 安裝開發工具
  poetry add --group dev pytest black jupyter

步驟 7: 檢查依賴樹
  poetry show --tree

步驟 8: 啟動環境
  poetry shell

步驟 9: 測試安裝
  python -c "import jieba; print('jieba OK')"
  python -c "import transformers; print('transformers OK')"
"""

print("📝 練習 1: 創建完整 NLP 專案")
print("="*60)
print(exercise1_steps)
print("="*60)

print("\n🎯 預期結果:")
print("  - 專案結構完整")
print("  - pyproject.toml 包含所有依賴")
print("  - poetry.lock 已生成")
print("  - .venv/ 虛擬環境已創建")
print("  - 所有套件可正常導入")

### 練習 2: 依賴版本管理實驗

實驗不同版本約束的效果:

In [ ]:
# 練習 2: 版本約束實驗
exercise2_steps = """
實驗 A: 使用 ^ (caret) 約束
  1. poetry add "requests@^2.28.0"
  2. 檢查 poetry.lock 中的實際版本
  3. 觀察允許的版本範圍: >= 2.28.0, < 3.0.0

實驗 B: 使用 ~ (tilde) 約束
  1. poetry add "requests@~2.28.0"
  2. 檢查版本範圍: >= 2.28.0, < 2.29.0
  3. 比較與 ^ 的差異

實驗 C: 精確版本
  1. poetry add "requests==2.28.0"
  2. 嘗試與其他套件組合,觀察衝突

實驗 D: 依賴衝突模擬
  1. poetry add "packageA" "packageB"
  2. 如果 A 需要 C >= 1.0, B 需要 C >= 2.0
  3. Poetry 會自動選擇 C >= 2.0
"""

print("🧪 練習 2: 依賴版本管理實驗")
print("="*60)
print(exercise2_steps)
print("="*60)

print("\n🔬 觀察重點:")
print("  1. 不同約束語法的實際效果")
print("  2. Poetry 如何自動解決依賴衝突")
print("  3. poetry.lock 如何鎖定精確版本")
print("  4. 使用 'poetry show --tree' 檢視依賴樹")

### 練習 3: 視覺化依賴關係

使用 Poetry 指令分析專案依賴:

In [ ]:
# 練習 3: 依賴分析工具
def analyze_dependencies():
    """分析並視覺化專案依賴"""
    
    print("📊 依賴分析工具\n")
    
    # 檢查是否在 Poetry 專案中
    if not Path('pyproject.toml').exists():
        print("⚠️ 未在 Poetry 專案目錄中")
        print("請先執行: poetry new my-project")
        return
    
    # 指令列表
    commands = [
        ('顯示直接依賴', 'poetry show --no-dev'),
        ('顯示開發依賴', 'poetry show --only dev'),
        ('顯示依賴樹', 'poetry show --tree'),
        ('檢查過時套件', 'poetry show --outdated'),
        ('檢查依賴完整性', 'poetry check'),
    ]
    
    print("可用的依賴分析指令:\n")
    for i, (desc, cmd) in enumerate(commands, 1):
        print(f"{i}. {desc}")
        print(f"   指令: {cmd}\n")
    
    # 模擬依賴樹
    dependency_tree = """
transformers 4.35.0
├── filelock *
├── huggingface-hub >=0.16.4,<1.0
│   ├── filelock *
│   ├── fsspec >=2023.5.0
│   ├── packaging >=20.9
│   ├── pyyaml >=5.1
│   ├── requests *
│   ├── tqdm >=4.42.1
│   └── typing-extensions >=3.7.4.3
├── numpy >=1.17
├── packaging >=20.0
├── pyyaml >=5.1
├── regex !=2019.12.17
├── requests *
├── tokenizers >=0.14,<0.19
└── tqdm >=4.27
    """
    
    print("\n📦 範例依賴樹 (transformers):")
    print(dependency_tree)
    
    print("\n💡 分析技巧:")
    print("  - 使用 --tree 查看完整依賴鏈")
    print("  - 使用 --outdated 檢查可更新套件")
    print("  - 使用 poetry check 驗證配置正確性")

analyze_dependencies()

---

## 📚 本課總結

### 核心要點回顧:

1. **Poetry 三大優勢:**
   - 自動依賴解析 (SAT Solver 算法)
   - 確定性鎖定 (poetry.lock)
   - 單一配置檔案 (pyproject.toml)

2. **核心配置:**
   - `virtualenvs.in-project = true` - 虛擬環境建在專案內
   - `installer.max-workers = 10` - 平行下載加速

3. **版本約束最佳實踐:**
   - 優先使用 `^` (caret) 語法
   - 避免使用 `==` (過於嚴格)
   - 依賴 poetry.lock 確保環境一致

4. **工作流程:**
   - `poetry add` - 安裝並更新配置
   - `poetry install` - 同步環境 (根據 lock 檔)
   - `poetry shell` - 啟動虛擬環境
   - `poetry show --tree` - 檢視依賴樹

---

## 🎯 下節預告

**CH01-02: 必要套件安裝**

我們將深入探討:
- NLP 套件生態系統全貌
- 中英文 NLP 工具安裝與配置
- 深度學習框架選擇 (PyTorch vs TensorFlow)
- 套件相容性檢查與衝突解決
- 模型檔案下載與快取管理

---

## 📖 延伸閱讀

1. **官方文檔:**
   - [Poetry 官方文檔](https://python-poetry.org/docs/)
   - [pyproject.toml 規範 (PEP 518)](https://peps.python.org/pep-0518/)
   - [語義化版本規範](https://semver.org/lang/zh-TW/)

2. **進階主題:**
   - [PubGrub 依賴解析算法](https://github.com/dart-lang/pub/blob/master/doc/solver.md)
   - [Poetry 插件系統](https://python-poetry.org/docs/plugins/)

3. **最佳實踐:**
   - [Python Packaging 使用指南](https://packaging.python.org/)
   - [Poetry CI/CD 整合範例](https://python-poetry.org/docs/ci/)

---

### 🙋 問題討論

有任何問題嗎?歡迎在討論區提問!

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)